# Exercise 2: Efficient Training with Resource Constraints and Capability Preservation 🚀
## 📘 Prerequisites
From Exercise 1, we learned how to create high-quality training sets using smart sampling techniques. Now we'll build upon this knowledge while considering resource constraints and model capabilities preservation.

## 🎯 New Challenges
### 1. Capability Preservation
Even with high-quality data, models can "forget" previously learned capabilities during fine-tuning. We need to:

* Maintain general knowledge
* Preserve essential capabilities
* Balance new and existing skills

Generally, this is done by introducing different collections, and in this case, we will add a well-known collection for generic knowledge to our preference collection

```
trl-lib/ultrafeedback_binarized
```

### 2. Resource Optimization
Training on full datasets, even high-quality ones, can be impractical due to:

* Workshop time constraints
* Cloud computing costs
* In an enterprise environment, respect the development cycle duration

### Install dependencies

In [ ]:
! pip install -r requirements.txt
! pip install flash-attn==2.7.3 --no-build-isolation

### Testing GPU
Please check if python recognize that you have GPU allocated, if not please go in `Settings`>`Accelerator`>`GPU T4 x 2` 

In [1]:
import os, sys
#from tensorflow.python.client import device_lib
repo_folder = os.getcwd().split('labs_AMLD25_Workshop')[0][:-1]+"/labs_AMLD25_Workshop/src" 
sys.path.append(repo_folder)

#device_lib.list_local_devices()

if you get two GPUs you can manually assign them using env variables. This step is optional since they should be automatically recognized by pytorch 

In [2]:
os.environ["WANDB_DISABLED"] = "true" ## turning off WandB logging
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"

In [24]:
import torch

from typing import Optional, List, Dict
import datasets
from datasets import (
    load_dataset, 
    load_from_disk, 
    DatasetDict,
    concatenate_datasets
)

from accelerate import Accelerator, PartialState
from transformers import AutoModelForCausalLM, AutoTokenizer

from trl import (
    ModelConfig,
    DPOTrainer,
    DPOConfig,
    TrlParser,
    get_kbit_device_map,
    get_peft_config,
    get_quantization_config,
)

from trlabs.rl.data import (
    get_datasets, 
    DataArguments
)

from trlabs.utils import *

from trl.trainer.utils import SIMPLE_CHAT_TEMPLATE

### Model Config

In [4]:
model_config = {
    "model_name_or_path": "Qwen/Qwen2-0.5B-Instruct",
    "torch_dtype": "bfloat16",
    "use_peft": True, 
    "lora_r": 64,        
    "lora_alpha": 32,    # Stronger updates
    "lora_dropout": 0.1, # Prevent overfitting
}


### Data Config
You can also leverage the preference dataset located in HF Hub `trl-lib/ultrafeedback_binarized`. 

In [5]:
data_params = {
  "dataset_name": "Mix 2",
  "dataset_mixer": {
    "trl-lib/ultrafeedback_binarized": 0.02, ## 1220 samples over 61000
    "./data/AMLD25_reuters_gentitle_1k": 1.,
  },
  "dataset_splits": ["train", "test"],
  "num_eval_samples": 100,
  "seed": 42
}

### Training Config

In [6]:
training_params =  {
    ## General
    "output_dir": f"{model_config['model_name_or_path'].split('/')[0].lower()}_ex2_output",
     "num_train_epochs": 1,
    "beta": 0.1,
    "eval_strategy": "steps",
    "eval_steps": 8,
    "per_device_train_batch_size": 1,
    "per_device_eval_batch_size": 1,
    "gradient_accumulation_steps": 8,
    #@ context length and max length (max_new_token = max_length - max_prompt_length)
    "max_length": 768,
    "max_prompt_length":512,
    ## Optimizer
    "optim": "adamw_torch",
    "learning_rate": 2.0e-4,
    "weight_decay": 0.001,
    "adam_epsilon": 1.0e-8,
    "adam_beta1": 0.9,
    "adam_beta2": 0.999,
    "max_grad_norm": 1.0,
    ## Scheduler ##
    "warmup_steps": 10,
    "lr_scheduler_type": "cosine",
    ## Logging
    "log_level": "info",
    "logging_first_step": True,
    "logging_steps": 10
}

## DPO Training Loop

If you launch the training as it is set up you will see that it will take about 1 hour!!! 

You can get the same results in much less time. 

### Let's Play with our two collection!
Now we have two collections, choosing **X** and **Y** fractions to balance the resulting contribution. But before you do that create collections with more smart sampling

```python
data_params = {
  "dataset_name": "Mix 2",
  "dataset_mixer": {
    "trl-lib/ultrafeedback_binarized": Y,
    "./data/AMLD25_reuters_gentitle_1k": X,
  },
  "dataset_splits": ["train", "test"],
  "num_eval_samples": 100,
  "seed": 42
}
```

In order to achive the best performance. 




##### Note:
A fraction of 0.1 for collection`"trl-lib/ultrafeedback_binarized"` means that we select 10% of the training size from this collection

In [23]:
from trlabs.rl.train import dpo

dpo(data_params, training_params, model_config)

## Give a look to the Model Generation

In [22]:
from trlabs.utils import dataset_creation, not_relevant_data

SYSTEM_PROMPT = 'You are an advanced AI system specialised in providing Reuters News title given a body text of the news.'
INSTRUCTION = "The title should be in capital letters and between 6 and 8 words in length. Please provide only the title as output and no other text or explanation."

dataset = load_dataset("ucirvine/reuters21578", 'ModApte', trust_remote_code=True)
dataset = dataset.filter(not_relevant_data).shuffle(seed=42).map(dataset_creation, fn_kwargs={"system_prompt": SYSTEM_PROMPT, "instruction": INSTRUCTION})

In [21]:
from trlabs.rl.eval import setup_model_and_lora, generate

index =33
prompt = dataset["test"][index]["system"]+dataset["test"][index]["messages"]

model, tokenizer = setup_model_and_lora(
    base_model_name = model_config["model_name_or_path"], 
    lora_path = training_params["output_dir"]
)

response = generate(prompt, model, tokenizer)
print(response)

#### Note: 
if you do not provide a lora_path you can check the base model output